# Homework 2

In [3]:
from fastembed import TextEmbedding

In [5]:
!python -m pip install -q "qdrant-client[fastembed]>=1.14.2"

## Q1. Embedding the query
Embed the query: 'I just discovered the course. Can I join now?'. Use the 'jinaai/jina-embeddings-v2-small-en' model.

You should get a numpy array of size 512.

What's the minimal value in this array?


```bash
docker pull qdrant/qdrant

docker run -p 6333:6333 -p 6334:6334 \
   -v "$(pwd)/qdrant_storage:/qdrant/storage:z" \
   qdrant/qdrant
```

In [1]:
from qdrant_client import QdrantClient, models
import numpy as np

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
model_handle = "jinaai/jina-embeddings-v2-small-en"
embedding_model = TextEmbedding(model_name=model_handle)

Fetching 5 files: 100%|██████████| 5/5 [00:01<00:00,  3.04it/s]


In [ ]:
from qdrant_client import QdrantClient, models

text_query='I just discovered the course. Can I join now?'

embeddings = list(embedding_model.embed([text_query]))
query_embedding = embeddings[0]

# Now you can access the embedding
print(f"Embedding size {query_embedding.shape}")  
print(f"Minimal element {np.min(query_embedding)}")

Embedding size (512,)
Minimal element -0.11726373885183883


## Cosine similarity

In [6]:
np.linalg.norm(query_embedding)

np.float64(1.0)

In [7]:
query_embedding.dot(query_embedding)

np.float64(1.0000000000000002)

## Q2. Cosine similarity with another vector
Now let's embed this document:

In [8]:
doc = 'Can I still join the course after the start date?'
embeddings_doc = list(embedding_model.embed([doc]))
query_embedding_doc = embeddings_doc[0]

print(f"Cosine similarity: {query_embedding.dot(query_embedding_doc) } ")

Cosine similarity: 0.9008528895674548 


## Q3. Ranking by cosine
For Q3 and Q4 we will use these documents:

In [10]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [11]:
embed_text = []

for doc in documents:
    embeddings_doc = list(embedding_model.embed([doc["text"]]))
    query_embedding_doc = embeddings_doc[0]
    embed_text.append(query_embedding_doc)

embed_text = np.array(embed_text)


Compute the embeddings for the text field, and compute the cosine between the query vector and all the documents.

What's the document index with the highest similarity? (Indexing starts from 0):

In [15]:
text_query='I just discovered the course. Can I join now?'

embeddings = list(embedding_model.embed([text_query]))
query_embedding = embeddings[0]

In [20]:
cos_sim =  embed_text.dot(query_embedding)  #np.dot(embed_text, query_embedding)

In [21]:
cos_sim

array([0.76296845, 0.81823782, 0.80853974, 0.71330788, 0.73044992])

In [22]:
print(f"The most similar document number is: {np.argmax(cos_sim)}")

print(f"The most similar document number is: {documents[np.argmax(cos_sim)]['question']}")

The most similar document number is: 1
The most similar document number is: Course - Can I follow the course after it finishes?


## Q4. Ranking by cosine, version two
Now let's calculate a new field, which is a concatenation of question and text:

full_text = doc['question'] + ' ' + doc['text']
Embed this field and compute the cosine between it and the query vector. What's the highest scoring document?

In [23]:
embed_text = []

for doc in documents:
    embeddings_doc = list(embedding_model.embed([doc['question'] + ' ' + doc['text']]))
    query_embedding_doc = embeddings_doc[0]
    embed_text.append(query_embedding_doc)

embed_text = np.array(embed_text)

In [24]:
cos_sim =  embed_text.dot(query_embedding)  #np.dot(embed_text, query_embedding)

In [25]:
print(f"The most similar document number is: {np.argmax(cos_sim)}")

print(f"The most similar document number is: {documents[np.argmax(cos_sim)]['question']}")

The most similar document number is: 0
The most similar document number is: Course - Can I still join the course after the start date?


## Q5. Selecting the embedding model
Now let's select a smaller embedding model. What's the smallest dimensionality for models in fastembed?

In [31]:
#import json

min_dim =np.inf

for model in TextEmbedding.list_supported_models():
    if model['dim'] < min_dim:
        min_dim = model['dim']
        min_model = model

print(f"Model with minimal dimension embedding: {min_model["model"]}, Dimensionality: {min_dim} ")


Model with minimal dimension embedding: BAAI/bge-small-en, Dimensionality: 384 


## Q6. Indexing with qdrant (2 points)
For the last question, we will use more documents.

We will select only FAQ records from our ml zoomcamp:

In [ ]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [32]:
model_handle = "BAAI/bge-small-en"
embedding_model = TextEmbedding(model_name=model_handle)

Fetching 5 files: 100%|██████████| 5/5 [00:01<00:00,  2.89it/s]


In [ ]:
embed_text = []

for doc in documents:
    embeddings_doc = list(embedding_model.embed([doc['question'] + ' ' + doc['text']]))
    query_embedding_doc = embeddings_doc[0]
    embed_text.append(query_embedding_doc)

embed_text = np.array(embed_text)

text_query='I just discovered the course. Can I join now?'

embeddings = list(embedding_model.embed([text_query]))
query_embedding = embeddings[0]

cos_sim =  embed_text.dot(query_embedding)  #np.dot(embed_text, query_embedding)

In [37]:
best_idx = np.argmax(cos_sim)
print(f"Highest score: {cos_sim[best_idx]}")

Highest score: 0.8507919907569885


Alternative implementation

Run in terminal 
docker run -d -p 6333:6333 -p 6334:6334 \
   -v "$(pwd)/qdrant_storage:/qdrant/storage:z" \
   --name qdrant-container \
   qdrant/qdrant

In [39]:
from qdrant_client import QdrantClient, models
import requests

# Connect to Qdrant (make sure it's running first!)
client = QdrantClient("http://localhost:6333")

# Get ML Zoomcamp documents only
docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []
for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue
    
    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

# Use the model from Q5 (BAAI/bge-small-en - 384 dimensions)
model_handle = "BAAI/bge-small-en"
collection_name = "ml-zoomcamp-qa"

# Create collection
client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=384,  # bge-small-en dimensions
        distance=models.Distance.COSINE
    )
)

# Add documents with question + text concatenated
points = []
for i, doc in enumerate(documents):
    text = doc['question'] + ' ' + doc['text']
    
    point = models.PointStruct(
        id=i,
        vector=models.Document(text=text, model=model_handle),
        payload={"text": doc['text']}
    )
    points.append(point)

# Insert points
client.upsert(collection_name=collection_name, points=points)

# Search with Q1 question
query = "I just discovered the course. Can I still join it?"
results = client.query_points(
    collection_name=collection_name,
    query=models.Document(text=query, model=model_handle),
    limit=1,
    with_payload=True
)

print(f"Highest score: {results.points[0].score}")

Highest score: 0.88934696
